# Load Qwen2.5-vl

In [31]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import torch
# default: Load the model on the available device(s)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="cpu"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    device_map="auto",
)

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")





Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
from dataset import load_dataset
dataset = load_dataset("ImageNet", split='val')

In [59]:
image = dataset[1]['image']
image

tensor([[[0.6000, 0.3451, 0.5059,  ..., 0.1216, 0.1255, 0.1176],
         [0.3137, 0.3020, 0.3216,  ..., 0.1255, 0.1333, 0.1255],
         [0.4706, 0.2000, 0.2941,  ..., 0.1255, 0.1412, 0.1373],
         ...,
         [0.2745, 0.2627, 0.2706,  ..., 0.0627, 0.0510, 0.0275],
         [0.2275, 0.2157, 0.2235,  ..., 0.0667, 0.0588, 0.0431],
         [0.1686, 0.1765, 0.1647,  ..., 0.0824, 0.0667, 0.0588]],

        [[0.7843, 0.4627, 0.6039,  ..., 0.1098, 0.1216, 0.1255],
         [0.4941, 0.3882, 0.3725,  ..., 0.1176, 0.1294, 0.1255],
         [0.6314, 0.2745, 0.3098,  ..., 0.1216, 0.1333, 0.1294],
         ...,
         [0.2745, 0.2667, 0.2745,  ..., 0.0588, 0.0510, 0.0275],
         [0.2275, 0.2235, 0.2314,  ..., 0.0627, 0.0588, 0.0431],
         [0.1725, 0.1843, 0.1725,  ..., 0.0824, 0.0627, 0.0549]],

        [[0.4392, 0.2706, 0.4157,  ..., 0.1059, 0.0941, 0.0863],
         [0.2235, 0.2353, 0.2902,  ..., 0.1098, 0.1098, 0.0980],
         [0.3843, 0.1647, 0.2196,  ..., 0.1059, 0.1137, 0.

In [60]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]
text = processor.apply_chat_template(messages)
inputs = processor(text=text, images=image, return_tensors="pt", do_rescale=False)

In [61]:
outputs = model.generate(**inputs, max_new_tokens=100)

In [62]:
processor.batch_decode(outputs, skip_special_tokens=True)

['system\nYou are a helpful assistant.\nuser\nDescribe this image.\n fishing']

In [67]:
print(inputs.keys())
print(inputs['input_ids'].shape)
print(inputs['attention_mask'].shape)
print(inputs['pixel_values'].shape)
print(inputs['image_grid_thw'])

dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
torch.Size([1, 143])
torch.Size([1, 143])
torch.Size([484, 1176])
tensor([[ 1, 22, 22]])


In [24]:
# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>
<|im_start|>assistant



In [25]:
import requests
from io import BytesIO
from PIL import Image
url = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
response = requests.get(url)
image_raw = Image.open(BytesIO(response.content))
print(image_raw.size)
import torchvision
toTensor = torchvision.transforms.ToTensor() # PIL Image to Tensor
image = toTensor(image_raw)
print(image.shape)

(2048, 1365)
torch.Size([3, 1365, 2048])


In [26]:
image = image.unsqueeze(0).repeat(2, 1, 1, 1)
print(image.shape)

torch.Size([2, 3, 1365, 2048])


In [27]:
inputs_gt = processor(text=text, images=image_raw, return_tensors="pt")
print(inputs_gt['pixel_values'].shape)
print(inputs_gt['pixel_values'].dtype)

torch.Size([14308, 1176])
torch.float32


In [28]:
inputs = processor.image_processor.preprocess(images=image, return_tensors="pt", do_rescale=False)
print(inputs['pixel_values'].shape)
print(inputs['pixel_values'].dtype)
print((inputs['pixel_values'][:14308,:] == inputs_gt['pixel_values']).all())

torch.Size([28616, 1176])
torch.float32
tensor(True)


In [ ]:
print(3*1365*2048)
print(14308* 1176)
16826208 // 2

8386560
16826208


8413104

In [22]:
import torch
a = torch.ones(6,2,2)
b = torch.ones(2,2,2)
a+b

RuntimeError: The size of tensor a (6) must match the size of tensor b (2) at non-singleton dimension 0

In [ ]:
constraint = Constraint()

In [33]:
inputs['pixel_values']

tensor([[ 0.8501,  0.8501,  0.8647,  ...,  1.3922,  1.3922,  1.3922],
        [ 0.9376,  0.9376,  0.9376,  ...,  1.4491,  1.4491,  1.4491],
        [ 0.9084,  0.9376,  0.9376,  ...,  1.4065,  1.4207,  1.4207],
        ...,
        [-0.1280, -0.1280, -0.1426,  ..., -0.2431, -0.2715, -0.3000],
        [-0.3324, -0.3324, -0.3032,  ..., -0.3000, -0.2715, -0.2857],
        [-0.3762, -0.4054, -0.4054,  ..., -0.4279, -0.4422, -0.4564]])

In [ ]:

from qwen_vl_utils import process_vision_info
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)
